In [1]:
# Perturbation implementation of stochastic growth model adapted from Fabrice Collard's Matlab code, http://fabcol.free.fr/
# tested in Julia 1.15.0
# this code is part of chapter 5, "Advanced Numerical Techniques" from the book: "Introduction to Quantitative Macroeconomics with Julia"
# Academic Press - Elsevier
# for comments, email at: petre(dot)caraiani(at)gmail(dot)com


#Pkg.add("ForwardDiff")
#Pkg.add("FiniteDiff")
#using ForwardDiff
#using FiniteDiff
using Calculus;
using LinearAlgebra;
using Plots;


#Setup the model
# number of backward, forward looking variables and shocks
nx = 2; # backward variables
ny = 1; # forward and static variables
ne = 1; # shocks

# calibration
alpha = 0.3;
sigma = 1.5;
beta  = 0.95;
delta = 0.1;
as    = 0;
rhoa  = 0.9;
eta1  = 0.01;
ETA   = [0;eta1];
s2    = 1;

#steady state
ksy=alpha*beta/(1-beta*(1-delta));
ysk=(1-beta*(1-delta))/(alpha*beta);
ys=ksy^(alpha/(1-alpha));
ks=ksy*ys;
cs=ys*(1-delta*ksy);

# vector of parameters 
param=[beta sigma alpha delta rhoa as]

# vector of steady state
xs=[ks as cs ks as cs];

xs=vec(xs)


6-element Vector{Float64}:
 2.6257456456982005
 0.0
 1.0733311148204927
 2.6257456456982005
 0.0
 1.0733311148204927

In [2]:
#compute Jacobian and Hessian matrices

#define the model
# k' a' c' k a c
# 1  2  3  4 5 6
#eq(1)= kp-exp(a)*k^alpha+c-(1-delta)*k; 
#eq(2)= ap-ra*a-(1-ra)*ab;
# Forward variables
#eq(3)= c^(-sigma)-beta*(cp^(-sigma))*(alpha*exp(ap)*(kp^(alpha-1))+1-delta);

#Jacobian
gr1 = Calculus.gradient(x -> x[1]-exp(x[5])*x[4]^alpha+x[6]-(1-delta)*x[4], xs);
gr2 = Calculus.gradient(x -> x[2]-rhoa*x[5]-(1-rhoa)*as, xs);
gr3 = Calculus.gradient(x -> x[6]^(-sigma)-beta*(x[3]^(-sigma))*(alpha*exp(x[2])*(x[1]^(alpha-1))+1-delta), xs);


#define the Jacobian
J =  [gr1'; gr2'; gr3']

3×6 Matrix{Float64}:
 1.0         0.0       0.0      -1.05263  -1.33591   1.0
 0.0         1.0       0.0       0.0      -0.9       0.0
 0.0347626  -0.130397  1.25677   0.0       0.0      -1.25677

In [3]:
#Hessian
h1 = hessian(x -> x[1]-exp(x[5])*x[4]^alpha+x[6]-(1-delta)*x[4], xs);
h2 = hessian(x -> x[2]-rhoa*x[5]-(1-rhoa)*as, xs);
h3 = hessian(x -> x[6]^(-sigma)-beta*(x[3]^(-sigma))*(alpha*exp(x[2])*(x[1]^(alpha-1))+1-delta), xs);

h1=vec(h1);
h2=vec(h2);
h3=vec(h3);
#define the Hessian
H =  [h1 h2 h3]

36×3 Matrix{Float64}:
  3.40502e-7  0.0  -0.0225065
  0.0         0.0   0.0347629
  0.0         0.0  -0.0485812
  0.0         0.0   0.0
  0.0         0.0   0.0
  0.0         0.0   0.0
  0.0         0.0   0.0347629
  0.0         0.0  -0.130397
  0.0         0.0   0.182232
  0.0         0.0   0.0
  ⋮                
 -0.152633    0.0   0.0
 -1.33591     0.0   0.0
  0.0         0.0   0.0
  0.0         0.0   0.0
  0.0         0.0   0.0
  0.0         0.0   0.0
  1.07432e-6  0.0   0.0
  0.0         0.0   0.0
  1.0543e-6   0.0   2.92727

In [4]:
#Solve the model

Hs=xs[1:nx];
Gs=xs[nx+1:nx+ny];

# Order 1
J0=J[:,1:nx+ny];
J1=-J[:,nx+ny+1:end];

r = schur(J0,J1)
s = r.S
t = r.T
q = r.Q  # So now q*a*z = s and q*b*z = t
z = r.Z


3×3 Matrix{Float64}:
  0.987655  0.156647  0.0
  0.0       0.0       1.0
 -0.156647  0.987655  0.0

In [5]:
#qzswitch procedure
nsize = size(s,1);
i = 1;

while i<=nsize-1;

   if 1+abs(t[i,i]*s[i+1,i+1])>1+abs(s[i,i]*t[i+1,i+1]);    
      A=s; B=t; Q=q; Z=z;   

      #[s,t,q,z] = qzswitch(i,s,t,q,z);
      a = A[i,i]; d = B[i,i]; b =A[i,i+1]; e = B[i,i+1]; c = A[i+1,i+1]; f = B[i+1,i+1]; 

      wz = [c*e-f*b, (c*d-f*a)'];
      xy = [(b*d-e*a)', (c*d-f*a)'];
      n = sqrt(wz'*wz);
      m = sqrt(xy'*xy);

      if n == 0
      return
      else
    wz = \(n,wz);
    xy = \(m,xy); 
    wz = [wz[1] wz[2]; -wz[2] wz[1]]
    xy = [xy[1] xy[2]; -xy[2] xy[1]]
    A[i:i+1,:] = xy*A[i:i+1,:];
    B[i:i+1,:] = xy*B[i:i+1,:];
    A[:,i:i+1] = A[:,i:i+1]*wz;
    B[:,i:i+1] = B[:,i:i+1]*wz;
    Z[:,i:i+1] = Z[:,i:i+1]*wz;
    Q[i:i+1,:] = xy*Q[i:i+1,:];
       end;

   if ~(i==1);i = i-2;end
   s=A; t=B; q=Q; z=Z;     
   end;         
   i=i+1;
   
end;


In [12]:
z21 = z[nx+1:end,1:nx];
z11 = z[1:nx,1:nx];

if rank(z11)<nx;
 error("Invertibility condition violated")
end

z11i = \(z11,Matrix{Float64}(I, nx, nx));
s11 = s[1:nx,1:nx];
t11 = t[1:nx,1:nx];

#if abs(t[nx,nx])>abs(s[nx,nx]) | abs(t[nx+1,nx+1])<abs(s[nx+1,nx+1]);
#warning("Wrong number of stable eigenvalues");
#end

2×2 Matrix{Float64}:
 -0.88438       0.690087
  5.50951e-17  -0.905667

In [7]:
#compute Gx and Hx
dyn = \(s11,t11);
Gx = z21*z11i;
Hx = z11*dyn*z11i;

tol=1e-6;
if maximum(maximum(imag(Gx)))<tol;
  Gx=real(Gx);
end
if maximum(maximum(imag(Gx)))<tol;
  Hx=real(Hx);
end


2×2 Matrix{Float64}:
 0.869055     0.729254
 2.88807e-17  0.9

In [8]:
# Computes Gxx and Hxx
Zx = [Hx;Gx*Hx;Matrix{Float64}(I, nx, nx);Gx];
Jxp= J[:,1:nx];
Jyp= J[:,nx+1:nx+ny];
Jx = J[:,nx+ny+1:2*nx+ny];
Jy = J[:,2*nx+ny+1:2*(nx+ny)];
XX1 = [kron((Jxp+Jyp*Gx),Matrix{Float64}(I, nx*nx, nx*nx)) kron(Jyp,kron(Hx',Hx'))+kron(Jy,Matrix{Float64}(I, nx*nx, nx*nx))];
XX0 = -kron(Zx',Zx')*H;
XX0 = vec(XX0);
HGXX= \(XX1,XX0);
Hxx = HGXX[1:nx*nx*nx];
if maximum(maximum(imag(Hxx)))<tol
   Hxx=real(Hxx);
end
Gxx = HGXX[nx*nx*nx+1:end];
if maximum(maximum(imag(Gxx)))<tol
   Gxx=real(Gxx);
end


4-element Vector{Float64}:
 -0.029888674719871767
  0.055536346951659425
  0.05553634695241073
  0.5026502804590514

In [9]:
# Computes Gss and Hss

SS0= 0;
for i=1:ne;
    Zs    =[ETA[:,i];Gx*ETA[:,i];zeros(nx,1);zeros(ny,1)];
    TEMP0 = -kron(Zs',Zs')*H;
    TEMP0 = vec(TEMP0);
    TEMP1 = -Jyp*kron(Matrix{Float64}(I, ny, ny),kron(ETA[:,i]',ETA[:,i]'))*Gxx;
    TEMP1 = vec(TEMP1);
    SS0   = SS0.+TEMP0+TEMP1;
end

SS1  = [Jxp+Jyp*Gx (Jyp+Jy)];
HGSS = \(SS1,SS0);
Hss  = HGSS[1:nx];
Gss  = HGSS[nx+1:nx+ny];
if maximum(maximum(imag(Hss)))<tol
   Hss=real(Hss);
end
if maximum(maximum(imag(Gss)))<tol
   Gss=real(Gss);
end

Hxx = reshape(Hxx,nx*nx,nx)';
Gxx = reshape(Gxx,nx*nx,ny)'

1×4 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.0298887  0.0555363  0.0555363  0.50265

In [10]:
# compute the variables
long  = 1000;
tronc = 100;
slong = long+tronc;
T     = tronc+1:slong;
e     = randn(ne,slong)*sqrt(s2);  

S1=zeros(nx,slong);
S2=zeros(nx,slong);
X1=zeros(ny,slong);
X2=zeros(ny,slong);
                
S1[:,1]=ETA*e[:,1]';
S2[:,1]=ETA*e[:,1]';
tmp=S2[:,1]*S2[:,1]';
tmp=vec(tmp);
X1[:,1]=Gx*S1[:,1];
X2[:,1]=X1[:,1]+0.5*Gxx*tmp[:];
for i=2:slong    
   S1[:,i]=Hx*S1[:,i-1]+ETA*e[:,i]';
   X1[:,i]=Gx*S1[:,i];
   S2[:,i]=S1[:,i]+0.5*Hxx*tmp+0.5*Hss*s2;
   tmp2=S2[:,i]*S2[:,i]';
   tmp2=vec(tmp2); 
   X2[:,i]=Gs+0.5*Gss*s2+X1[:,i]+0.5*Gxx*tmp2;   
   X1[:,i]=Gs+X1[:,i];
    end;


In [11]:
#plot series

plotly() # Choose the Plotly.jl backend for web interactivity
#plot(k',c',linewidth=1,label="Consumption",title="Consumption vs capital stock")
plot(X2[1,2:1000],linewidth=1,label="Consumption")

┌ Warning: Failed to load integration with PlotlyBase & PlotlyKaleido.
│   exception = (ArgumentError("Package PlotlyKaleido not found in current path.\n- Run `import Pkg; Pkg.add(\"PlotlyKaleido\")` to install the PlotlyKaleido package."), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007ffdc3725dd4, Ptr{Nothing} @0x00007ffdc3029364, Ptr{Nothing} @0x00007ffdddfa96fe, Ptr{Nothing} @0x00007ffdc342be56, Ptr{Nothing} @0x00007ffdc3064314, Ptr{Nothing} @0x00007ffdddfda092, Ptr{Nothing} @0x00007ffdddfdbf66, Ptr{Nothing} @0x00007ffdddfbdb6a, Ptr{Nothing} @0x00007ffdddfbe39e, Base.InterpreterIP in top-level CodeInfo for Main at statement 1, Ptr{Nothing} @0x00007ffdddfdb2b8, Ptr{Nothing} @0x00007ffdddfdca36, Ptr{Nothing} @0x000002e285756ffe, Ptr{Nothing} @0x000002e2857577f8, Ptr{Nothing} @0x000002e285757946, Ptr{Nothing} @0x000002e285757974, Ptr{Nothing} @0x00007ffdddfbccfa, Ptr{Nothing} @0x00007ffdddfbc7f4, Ptr{Nothing} @0x00007ffdddfbd440, Ptr{Nothing} @0x00007ffdddfbe39e, Base.In